<a href="https://colab.research.google.com/github/balajianbalagan/kavach23/blob/main/FRAUDBUSTER_AI_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PRE-PROCESSING STEP 1

import pandas as pd
import numpy as np
import json
import csv
from random import randint

#paysim dataset
data_path = "PS_20174392719_1491204439457_log.csv"

print("DATA PREPROCESSING STAGE 1")

#read dataset
X = pd.read_csv(data_path)
X = X.to_numpy()

nameOrigCol = 3
nameDestCol = 6
nameOrig = []
nameDest = []
nameCount = {}
namesWithMoreThanOneOccurrence = [] #to find accounts that occur more than once

for name in X[:,nameOrigCol]:
    if nameCount.get(name,-1) == -1:
        nameOrig.append(name)

        nameCount[name] = 1

    else:
        nameCount[name] += 1
        namesWithMoreThanOneOccurrence.append(name)

for name in X[:,nameDestCol]:
    if nameCount.get(name,-1) == -1: #add new account
        nameDest.append(name)
        nameCount[name] = 1

    else:
        nameCount[name] += 1 #increase count
        namesWithMoreThanOneOccurrence.append(name)

#calculating the median value
countArr = []
count = 0
for attr, value in nameCount.items():
    if value>40:
        countArr.append(value)
        count += 1
median = np.median(countArr)

print(f"Median : {median}")

#Filtering data based on median transactions count
csv_main_data = []

for i in range(X.shape[0]):
    if nameCount.get(X[i,3],-1) > median or nameCount.get(X[i,6],-1) > median:
        csv_main_data.append(X[i,:])

#write filtered data into a new file
with open("data_processed/filtered_data.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(csv_main_data)

print("DATA PREPROCESSING 1 DONE")

DATA PREPROCESSING 1 STAGE
Median : 45.0
DATA PREPROCESSING 1 DONE


In [ ]:
#PRE-PROCESSING STEP 2

import pandas
import json
import numpy as np
import csv
from random import randint

print("DATA PREPROCESSING STAGE 2")

#read filtered data from previous step
data_path = "data_processed/filtered_data.csv"

X = pandas.read_csv(data_path)
X = X.to_numpy()

#columns
csv_dataset_primary = []
step = 0
trans_type = 1
amount = 2
nameOrig = 3
oldbalanceOrg = 4
nameDest = 6
oldbalanceDest = 7
accountType = 8
isFraud = 9
isFlaggedFraud = 10

#change the labels
transfer = ["WIRE_IN", "WIRE_OUT"]
for i in range(X.shape[0]):
	arr = []
	arr.append(X[i,step])
	if X[i,trans_type] =="PAYMENT":
		arr.append("CREDIT")
	elif X[i,trans_type] =="TRANSFER":
		arr.append(transfer[randint(0,1)])
	else:
		arr.append(X[i,trans_type])
	arr.append(X[i,amount])
	arr.append(X[i,nameOrig])
	arr.append(X[i,oldbalanceOrg])
	arr.append(X[i,nameDest])
	arr.append(X[i,oldbalanceDest])
	if X[i,trans_type] == "TRANSFER":
		arr.append("FOREIGN")
	else:
		arr.append("DOMESTIC")

	arr.append(X[i,isFraud])
	arr.append(X[i,isFlaggedFraud])

	csv_dataset_primary.append(arr)

#converting to df and storing data in csv file
columns=['step','trans_type','amount','nameOrig','oldbalanceOrg',
        'nameDest','oldbalanceDest','accountType','isFraud','isFlaggedFraud']

data_primary = pandas.DataFrame(csv_dataset_primary, columns=columns)

data_primary.to_csv('data_processed/filtered_data_2.csv', index=False)

print("DATA PREPROCESSING 2 DONE")

DATA PREPROCESSING STAGE 2
DATA PREPROCESSING 2 DONE


In [ ]:
import pandas
import json
import numpy as np
import csv
from random import randint

print("DATA PREPROCESSING STAGE 3")
data_path = "data_processed/filtered_data_2.csv"

X = pandas.read_csv(data_path)
X = X.to_numpy()

#columns
step = 0
trans_type = 1
amount = 2
nameOrig = 3
oldbalanceOrg = 4
nameDest = 5
oldbalanceDest = 6
accountType = 7
isFraud = 8

#new columns
entity = 0
#to calculate the amount credited and debited in an account for every 30,60,90 steps
incommingDomestic30 = 1
incommingDomestic60 = 2
incommingDomestic90 = 3
outgoingDomestic30 = 4
outgoingDomestic60 = 5
outgoingDomestic90 = 6
incommingForeign30 = 7
incommingForeign60 = 8
incommingForeign90 = 9
outgoingForeign30 = 10
outgoingForeign60 = 11
outgoingForeign90 = 12
#to find the count of transactions occuring for account every 30,60,90 steps
incoming_domestic_count_30 = 13
incoming_domestic_count_60 = 14
incoming_domestic_count_90 = 15
outgoing_domestic_count_30 = 16
outgoing_domestic_count_60 = 17
outgoing_domestic_count_90 = 18
incoming_foreign_count_30 = 19
incoming_foreign_count_60 = 20
incoming_foreign_count_90 = 21
outgoing_foreign_count_30 = 22
outgoing_foreign_count_60 = 23
outgoing_foreign_count_90 = 24
#find the balance differenceevery 30,60,90 steps
balance_difference_30 = 25
balance_difference_60 = 26
balance_difference_90 = 27
isFraudSec = 28

csv_dataset_secondary = []
entities_pos = {}
enititesDict = {} #stores data of entities(bank account)

#Creating New Features Using Transaction History

def getSecRow(entity):
	return [entity,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(X.shape[0]):
	source_entity = X[i,nameOrig]
	dest_entity = X[i,nameDest]

	source_pos = entities_pos.get(source_entity,-1)
	if source_pos == -1:
		pos = len(csv_dataset_secondary)
		entities_pos[source_entity] = pos
		source_pos = pos

		row = getSecRow(source_entity)

		csv_dataset_secondary.append(row)

	dest_pos = entities_pos.get(dest_entity,-1)
	if dest_pos == -1:
		pos = len(csv_dataset_secondary)
		entities_pos[dest_entity] = pos
		dest_pos = pos

		row = getSecRow(dest_entity)
		
		csv_dataset_secondary.append(row)

	transferAmountSource = 0
	transferAmountDest = 0

	if X[i,trans_type] == "CASH_IN" or X[i,trans_type] == "CREDIT": #calc incoming and outgoing amount - domestic
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][incommingDomestic30] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingDomestic30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][incommingDomestic60] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingDomestic60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][incommingDomestic90] += X[i,amount] 
			csv_dataset_secondary[dest_pos][outgoingDomestic90] += X[i,amount]

		transferAmountSource = X[i,amount]
		transferAmountDest = -1*X[i,amount]

	if X[i,trans_type] == "CASH_OUT" or X[i,trans_type] == "DEBIT": #calc incoming and outgoing amount - domestic
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][outgoingDomestic30] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingDomestic30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][outgoingDomestic60] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingDomestic60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][outgoingDomestic90] += X[i,amount] 
			csv_dataset_secondary[dest_pos][incommingDomestic90] += X[i,amount]

		transferAmountSource = -1*X[i,amount]
		transferAmountDest = X[i,amount]

	if X[i,trans_type] == "WIRE_IN": #calc incoming and outgoing amount - foreign
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][incommingForeign30] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingForeign30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][incommingForeign60] += X[i,amount]
			csv_dataset_secondary[dest_pos][outgoingForeign60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][incommingForeign90] += X[i,amount] 
			csv_dataset_secondary[dest_pos][outgoingForeign90] += X[i,amount]

		transferAmountSource = X[i,amount]
		transferAmountDest = -1*X[i,amount]

	if X[i,trans_type] == "WIRE_OUT": #calc incoming and outgoing amount - foreign
		if X[i,step] <=30:
			csv_dataset_secondary[source_pos][outgoingForeign30] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingForeign30] += X[i,amount]
		if X[i,step] <=60:
			csv_dataset_secondary[source_pos][outgoingForeign60] += X[i,amount]
			csv_dataset_secondary[dest_pos][incommingForeign60] += X[i,amount]
		if X[i,step] <=90:
			csv_dataset_secondary[source_pos][outgoingForeign90] += X[i,amount] 
			csv_dataset_secondary[dest_pos][incommingForeign90] += X[i,amount]

		transferAmountSource = -1*X[i,amount]
		transferAmountDest = X[i,amount]

	if enititesDict.get(source_entity,-1) == -1:
		enititesDict[source_entity] = {
										'day1Bal': X[i,oldbalanceOrg],
										'day30Bal': 0,
										'day60Bal': 0,
										'day90Bal': 0,
										'countIncomingDomestic30': 0,
										'countOutgoingDomestic30': 0,
										'countIncomingDomestic60': 0,
										'countOutgoingDomestic60': 0,
										'countIncomingDomestic90': 0,
										'countOutgoingDomestic90': 0,
										'countIncomingForeign30': 0,
										'countOutgoingForeign30': 0,
										'countIncomingForeign60': 0,
										'countOutgoingForeign60': 0,
										'countIncomingForeign90': 0,
										'countOutgoingForeign90': 0
										}

	if enititesDict.get(dest_entity,-1) == -1:
		enititesDict[dest_entity] = {
										'day1Bal': X[i,oldbalanceDest],
										'day30Bal': 0,
										'day60Bal': 0,
										'day90Bal': 0,
										'countIncomingDomestic30': 0,
										'countOutgoingDomestic30': 0,
										'countIncomingDomestic60': 0,
										'countOutgoingDomestic60': 0,
										'countIncomingDomestic90': 0,
										'countOutgoingDomestic90': 0,
										'countIncomingForeign30': 0,
										'countOutgoingForeign30': 0,
										'countIncomingForeign60': 0,
										'countOutgoingForeign60': 0,
										'countIncomingForeign90': 0,
										'countOutgoingForeign90': 0
									}

	incomingForSource = ["CASH_IN","CREDIT","WIRE_IN"]
	incomingForDest = ["CASH_OUT","DEBIT","WIRE_OUT"]
	outgoingForDest = incomingForSource
	outgoingForSource = incomingForDest

	if X[i,step]<=30: #calc count and balance
		enititesDict[source_entity]['day30Bal'] = transferAmountSource+X[i,oldbalanceOrg]
		enititesDict[dest_entity]['day30Bal'] = transferAmountDest+X[i,oldbalanceDest]
		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForSource:
				enititesDict[source_entity]['countIncomingForeign30'] += 1
			else:
				enititesDict[source_entity]['countOutgoingForeign30'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[source_entity]['countIncomingDomestic30'] += 1
			else:
				enititesDict[source_entity]['countOutgoingDomestic30'] += 1


		if X[i,accountType] == "FOREIGN": #calc count and balance
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingForeign30'] += 1
			else:

				enititesDict[dest_entity]['countOutgoingForeign30'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingDomestic30'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingDomestic30'] += 1

	if X[i,step]<=60: #calc count and balance
		enititesDict[source_entity]['day60Bal'] = transferAmountSource+X[i,oldbalanceOrg]
		enititesDict[dest_entity]['day60Bal'] = transferAmountDest+X[i,oldbalanceDest]
		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForSource:
				enititesDict[source_entity]['countIncomingForeign60'] += 1
			else:
				enititesDict[source_entity]['countOutgoingForeign60'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[source_entity]['countIncomingDomestic60'] += 1
			else:
				enititesDict[source_entity]['countOutgoingDomestic60'] += 1

		if X[i,accountType] == "FOREIGN": #calc count and balance
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingForeign60'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingForeign60'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingDomestic60'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingDomestic60'] += 1

	if X[i,step]<=90: #calc count and balance
		enititesDict[source_entity]['day90Bal'] = transferAmountSource+X[i,oldbalanceOrg]
		enititesDict[dest_entity]['day90Bal'] = transferAmountDest+X[i,oldbalanceDest]
		if X[i,accountType] == "FOREIGN":
			if X[i,trans_type] in incomingForSource:
				enititesDict[source_entity]['countIncomingForeign90'] += 1
			else:
				enititesDict[source_entity]['countOutgoingForeign90'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[source_entity]['countIncomingDomestic90'] += 1
			else:
				enititesDict[source_entity]['countOutgoingDomestic90'] += 1

		if X[i,accountType] == "FOREIGN": #calc count and balance
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingForeign90'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingForeign90'] += 1
		else:
			if X[i,trans_type] in incomingForDest:
				enititesDict[dest_entity]['countIncomingDomestic90'] += 1
			else:
				enititesDict[dest_entity]['countOutgoingDomestic90'] += 1


	#store the data in df
	csv_dataset_secondary[source_pos][balance_difference_30] = enititesDict[source_entity]['day30Bal'] - enititesDict[source_entity]['day1Bal']
	csv_dataset_secondary[source_pos][balance_difference_60] = enititesDict[source_entity]['day60Bal'] - enititesDict[source_entity]['day1Bal']
	csv_dataset_secondary[source_pos][balance_difference_90] = enititesDict[source_entity]['day90Bal'] - enititesDict[source_entity]['day1Bal'] 



	csv_dataset_secondary[source_pos][incoming_domestic_count_30] = enititesDict[source_entity]['countIncomingDomestic30']
	csv_dataset_secondary[source_pos][outgoing_domestic_count_30] = enititesDict[source_entity]['countOutgoingDomestic30']
	csv_dataset_secondary[source_pos][incoming_domestic_count_60] = enititesDict[source_entity]['countIncomingDomestic60']
	csv_dataset_secondary[source_pos][outgoing_domestic_count_60] = enititesDict[source_entity]['countOutgoingDomestic60']
	csv_dataset_secondary[source_pos][incoming_domestic_count_90] = enititesDict[source_entity]['countIncomingDomestic90']
	csv_dataset_secondary[source_pos][outgoing_domestic_count_90] = enititesDict[source_entity]['countOutgoingDomestic90'] 
	csv_dataset_secondary[source_pos][incoming_foreign_count_30] = enititesDict[source_entity]['countIncomingForeign30']
	csv_dataset_secondary[source_pos][outgoing_foreign_count_30] = enititesDict[source_entity]['countOutgoingForeign30']
	csv_dataset_secondary[source_pos][incoming_foreign_count_60] = enititesDict[source_entity]['countIncomingForeign60']
	csv_dataset_secondary[source_pos][outgoing_foreign_count_60] = enititesDict[source_entity]['countOutgoingForeign60']
	csv_dataset_secondary[source_pos][incoming_foreign_count_90] = enititesDict[source_entity]['countIncomingForeign90']
	csv_dataset_secondary[source_pos][outgoing_foreign_count_90] = enititesDict[source_entity]['countOutgoingForeign90'] 

	csv_dataset_secondary[source_pos][isFraudSec] = csv_dataset_secondary[source_pos][isFraudSec] or X[i,isFraud] 

	csv_dataset_secondary[dest_pos][incoming_domestic_count_30] = enititesDict[dest_entity]['countIncomingDomestic30']
	csv_dataset_secondary[dest_pos][outgoing_domestic_count_30] = enititesDict[dest_entity]['countOutgoingDomestic30']
	csv_dataset_secondary[dest_pos][incoming_domestic_count_60] = enititesDict[dest_entity]['countIncomingDomestic60']
	csv_dataset_secondary[dest_pos][outgoing_domestic_count_60] = enititesDict[dest_entity]['countOutgoingDomestic60']
	csv_dataset_secondary[dest_pos][incoming_domestic_count_90] = enititesDict[dest_entity]['countIncomingDomestic90']
	csv_dataset_secondary[dest_pos][outgoing_domestic_count_90] = enititesDict[dest_entity]['countOutgoingDomestic90'] 
	csv_dataset_secondary[dest_pos][incoming_foreign_count_30] = enititesDict[dest_entity]['countIncomingForeign30']
	csv_dataset_secondary[dest_pos][outgoing_foreign_count_30] = enititesDict[dest_entity]['countOutgoingForeign30']
	csv_dataset_secondary[dest_pos][incoming_foreign_count_60] = enititesDict[dest_entity]['countIncomingForeign60']
	csv_dataset_secondary[dest_pos][outgoing_foreign_count_60] = enititesDict[dest_entity]['countOutgoingForeign60']
	csv_dataset_secondary[dest_pos][incoming_foreign_count_90] = enititesDict[dest_entity]['countIncomingForeign90']
	csv_dataset_secondary[dest_pos][outgoing_foreign_count_90] = enititesDict[dest_entity]['countOutgoingForeign90'] 


columns = ['entity','incoming_domestic_amount_30','incoming_domestic_amount_60','incoming_domestic_amount_90', 
                     'outgoing_domestic_amount_30','outgoing_domestic_amount_60','outgoing_domestic_amount_90',
                     'incoming_foreign_amount_30','incoming_foreign_amount_60','incoming_foreign_amount_90',
                     'outgoing_foreign_amount_30','outgoing_foreign_amount_60','outgoing_foreign_amount_90',
                     'incoming_domestic_count_30','incoming_domestic_count_60','incoming_domestic_count_90',
                     'outgoing_domestic_count_30','outgoing_domestic_count_60','outgoing_domestic_count_90',
                     'incoming_foreign_count_30','incoming_foreign_count_60','incoming_foreign_count_90',
                     'outgoing_foreign_count_30','outgoing_foreign_count_60','outgoing_foreign_count_90',
                     'balance_difference_30','balance_difference_60','balance_difference_90','isFraud']


data_secondary = pandas.DataFrame(csv_dataset_secondary, columns=columns)
data_secondary.to_csv('data_processed/filtered_data_3.csv', index=False)
print("DATA PREPROCESSING 3 DONE")

DATA PREPROCESSING STAGE 3
DATA PREPROCESSING 3 DONE
